## Basic Co-Occurrence Analysis of Human-Cell-Atlas Pancreas Topograhic Maps

Data from Tosti et al, 2021

In [1]:
%load_ext autoreload
%autoreload 2


In [11]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [44]:
rel_kernel_path = 'test/ISS-pancreas' #"."


In [17]:
# Define the cluster labels of the pancreas tissue matrix:

cluster_labels = ['background',
 'Acinar-i',
 'Acinar-s',
 'Acinar-REG+',
 'Activated Stellate',
 'Quiescent Stellate',
 'Alpha',
 'Beta',
 'Gamma (PP)',
 'Delta',
 'Ductal',
 'MUC5B+ Ductal (PDG)',
 'MHC Class II',
 'Schwan',
#  'B cell',
#  'T cell',
 'Endothelial']

In [45]:
n_sample = 1

# Load the pancreas tissue matrix:

tissue_matrix = np.load(os.path.join( *rel_kernel_path.split('/'), 'data','sample_matrices.npz'))['sample_1']

In [47]:

plt.imshow(tissue_matrix.argmax(axis=0))

NameError: name 'argmax' is not defined